# Import Libraries

In [159]:
import pandas as pd
import pandas as pd
import re

# Load Data

In [160]:
# Load the datasets with specific data types to ensure correct handling
df_asset = pd.read_csv('Assessment Data Asset Dummy.csv', dtype={'Funcloc': str})
df_master = pd.read_csv('City Indonesia.csv', dtype={'RegionalCode': str, 'CityCode': str})

print("Data Loaded Successfully.")
print(f"Asset Data Rows: {len(df_asset)}")
print(f"Master City Rows: {len(df_master)}")

Data Loaded Successfully.
Asset Data Rows: 4908
Master City Rows: 514


# Data Validation

## Check for Required Fields

In [161]:
print("--- DATA VALIDATION REPORT ---\n")

# --- 1a. Check for required fields ---
# We check if 'Funcloc' is null or empty
missing_funcloc = df_asset['Funcloc'].isna().sum()
print(f"1a. Missing 'Funcloc' Records: {missing_funcloc}")

# --- 1b. Validate data formats ---
# We check if 'Alamat4' (Kabupaten/Kota) is present and is a string
# Logic: Count rows where Alamat4 is NOT a string or is null
invalid_format_alamat = df_asset[~df_asset['Alamat4'].apply(lambda x: isinstance(x, str))].shape[0]
print(f"1b. Rows with invalid/missing 'Alamat4' format: {invalid_format_alamat}")

# --- 1c. Verify numeric values  ---
# Funcloc should be numeric digits. We check for non-digit characters.
# We use a regex to find any row that contains non-digits.
non_numeric_funcloc = df_asset[~df_asset['Funcloc'].str.isdigit().fillna(False)]
print(f"1c. Non-numeric 'Funcloc' values found: {len(non_numeric_funcloc)}")
if not non_numeric_funcloc.empty:
    print(f"    Examples: {non_numeric_funcloc['Funcloc'].head(3).tolist()}")

# --- 1d. Validate referential integrity  ---
# Check how many 'Alamat4' values exist in the Master Data 'City' column.
# Note: We expect many failures here because we haven't done Data Cleaning (Task 2) yet.
unique_asset_cities = df_asset['Alamat4'].unique()
unique_master_cities = df_master['City'].unique()

# Calculate intersection
matches = df_asset['Alamat4'].isin(unique_master_cities).sum()
failures = len(df_asset) - matches

print(f"1d. Referential Integrity Check:")
print(f"    - Total Asset Records: {len(df_asset)}")
print(f"    - Direct Matches with Master Data: {matches}")
print(f"    - Mismatches (Integrity Failures): {failures}")
print("\nObservation: The high number of mismatches in (1d) confirms that Data Cleaning (Task 2) is required to standardize the city names.")

--- DATA VALIDATION REPORT ---

1a. Missing 'Funcloc' Records: 0
1b. Rows with invalid/missing 'Alamat4' format: 0
1c. Non-numeric 'Funcloc' values found: 0
1d. Referential Integrity Check:
    - Total Asset Records: 4908
    - Direct Matches with Master Data: 0
    - Mismatches (Integrity Failures): 4908

Observation: The high number of mismatches in (1d) confirms that Data Cleaning (Task 2) is required to standardize the city names.


# Data Cleaning

## Align alamat4 with master data

In [162]:
print("--- TASK 2: DATA CLEANING ---")

# 2e. Transform column "alamat4" to align with master data
def normalize_city_name(text):
    if pd.isna(text):
        return ""
    
    # 1. Convert to uppercase
    text = str(text).upper()
    text = re.sub(r'\(.*?\)', '', text)

    # Specific Fixes (Typos & Abbreviations)
    # Fix: PAHUWATO -> POHUWATO
    text = text.replace("PAHUWATO", "POHUWATO")

    # Fix: PANGKAJENE -> PANGKAJENE KEPULAUAN
    # Ensure we don't double up if it's already correct
    if "PANGKAJENE" in text and "KEPULAUAN" not in text:
        text = text.replace("PANGKAJENE", "PANGKAJENE KEPULAUAN")
    
    # Fix: KEPULAUANSIAUTAGULANDAN -> KEPULAUAN SIAU TAGULANDANG BIARO
    # The input might be "KEPULAUAN SIAU TAGULANDAN" (missing G and BIARO)
    if "SIAU TAGULANDAN" in text and "BIARO" not in text:
         text = text.replace("TAGULANDAN", "TAGULANDANG BIARO")
    
    # Fix: KABTANGERANG -> Remove "KAB."
    text = text.replace("KAB.", "")
    
    # Fix KUTAI KERTANEGARA -> KUTAI KARTANEGARA
    text = text.replace("KERTANEGARA", "KARTANEGARA")
    
    # Fix OKU -> OGAN KOMERING ULU
    text = re.sub(r'\bOKU\b', 'OGAN KOMERING ULU', text)
    
    # 2. Remove administrative prefixes/suffixes first
    # This ensures "KABUPATEN BOGOR" becomes "BOGOR"
    text = text.replace("KABUPATEN", "")
    text = text.replace("KOTA ADM", "") 
    text = text.replace("KOTA", "")
    text = text.replace("ADM", "")
    
    # 3. YOUR IDEA: Remove spaces and any character that is NOT A-Z
    # Regex explanation: [^A-Z] matches any character that is NOT A-Z. 
    # We replace them with empty string.
    text = re.sub(r'[^A-Z]', '', text)
    
    return text

# Apply the enhanced cleaning function
df_asset['City_Join_Key'] = df_asset['Alamat4'].apply(normalize_city_name)
df_master['City_Join_Key'] = df_master['City'].apply(normalize_city_name)


# Check match improvement
clean_matches = df_asset['City_Join_Key'].isin(df_master['City_Join_Key']).sum()
print(f"\nMatch Status:")
print(f"    - Matches found: {clean_matches} out of {len(df_asset)}")
print(f"    - Invalid/Unmapped: {len(df_asset) - clean_matches}")

--- TASK 2: DATA CLEANING ---

Match Status:
    - Matches found: 4900 out of 4908
    - Invalid/Unmapped: 8


# Data Transformation

In [163]:
print("\n--- TASK 3: DATA TRANSFORMATION ---")

# 1. Merge Asset Data with Master Data to get Codes (AAA and BB)
# We use the 'City_Join_Key' created in Task 2
# We keep 'left' join initially to track which rows fail (for Task 4 later)
df_merged = pd.merge(
    df_asset,
    df_master[['City_Join_Key', 'CityCode', 'RegionalCode']],
    on='City_Join_Key',
    how='left'
)

# 2. Filter for Valid Records
# We can only generate IDs for records that successfully matched a city.
df_valid = df_merged.dropna(subset=['CityCode', 'RegionalCode']).copy()

# 3. Define Order (Task 3b)
# "order_by funcloc group_by city_code"
df_valid.sort_values(by=['CityCode', 'Funcloc'], inplace=True)

# 4. Generate Sequence Number 'CCC' (Task 3d.iv)
# This creates a counter (1, 2, 3...) that resets for each unique CityCode
df_valid['Sequence_Num'] = df_valid.groupby('CityCode').cumcount() + 1

# 5. Construct 'Internal Site ID' (Task 3d)
# AAA = City Code
aaa_part = df_valid['CityCode']

# BB = Regional Code (Ensure it has 2 digits, e.g., 5 -> '05')
bb_part = df_valid['RegionalCode'].astype(str).str.zfill(2)

# CCC = Sequence Number (Ensure it has 3 digits, e.g., 1 -> '001')
ccc_part = df_valid['Sequence_Num'].astype(str).str.zfill(3)

# Combine them: AAA-BB-CCC
df_valid['Internal Site ID'] = aaa_part + '-' + bb_part + '-' + ccc_part

# 6. Final Column Selection
# Select only the columns requested for the final result
final_output = df_valid[['Internal Site ID', 'Alamat4', 'Funcloc']]

print("Transformation complete.")
print(f"Generated Internal Site IDs for {len(final_output)} valid assets.")
print("\nSample Result:")
print(final_output.head(10).to_string(index=False))


--- TASK 3: DATA TRANSFORMATION ---
Transformation complete.
Generated Internal Site IDs for 5629 valid assets.

Sample Result:
Internal Site ID        Alamat4      Funcloc
      ADL-10-001 KONAWE SELATAN 100000001078
      ADL-10-002 KONAWE SELATAN 100000001079
      ADL-10-003 KONAWE SELATAN 100000001080
      ADL-10-004 KONAWE SELATAN 100000001081
      ADL-10-005 KONAWE SELATAN 100000001082
      ADL-10-006 KONAWE SELATAN 100000001995
      ADL-10-007 KONAWE SELATAN 100000005016
      ADL-10-008 KONAWE SELATAN 100000005045
      ADL-10-009 KONAWE SELATAN 100000012313
      ADL-10-010 KONAWE SELATAN 100000018777


# Error Handling


In [164]:
print("\n--- TASK 4: ERROR HANDLING ---")

# 1. Identify Invalid Records
invalid_records = df_merged[df_merged['CityCode'].isna()].copy()

# 2. Create Error Log
error_log = invalid_records[['Funcloc', 'Alamat4', 'City_Join_Key']].copy()
error_log['Error_Description'] = 'City not found in Master Data'

# 3. Display the Log
print(f"Total Invalid Records: {len(error_log)}")

if not error_log.empty:
    print("\n--- Error Log (First 10 Records) ---")
    print(error_log.head(10).to_string(index=False))
    
    # Optional: Analyze common unmatched cities to see if we can improve cleaning logic
    print("\n--- Top 5 Unmatched Cities ---")
    print(invalid_records['City_Join_Key'].value_counts().head(5))
else:
    print("Success: No invalid records found.")


--- TASK 4: ERROR HANDLING ---
Total Invalid Records: 8

--- Error Log (First 10 Records) ---
     Funcloc Alamat4 City_Join_Key             Error_Description
100000008266 JAKARTA       JAKARTA City not found in Master Data
100000008268 JAKARTA       JAKARTA City not found in Master Data
100000008274 JAKARTA       JAKARTA City not found in Master Data
100000008277 JAKARTA       JAKARTA City not found in Master Data
100000008279 JAKARTA       JAKARTA City not found in Master Data
100000008281 JAKARTA       JAKARTA City not found in Master Data
100000008284 JAKARTA       JAKARTA City not found in Master Data
100000008285 JAKARTA       JAKARTA City not found in Master Data

--- Top 5 Unmatched Cities ---
City_Join_Key
JAKARTA    8
Name: count, dtype: int64
